In [1]:
import numpy as np 
import pandas as pd 
from pandas import DataFrame 

import matplotlib.pyplot as plt 
import seaborn as sns
import itertools
%matplotlib inline
from matplotlib.pyplot import figure
import matplotlib as mpl
import matplotlib.font_manager as fm
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.family"] = 'Malgun Gothic' 
plt.rcParams["font.size"] = 10
%matplotlib inline 


from datetime import datetime, timedelta 
from statsmodels.tsa.arima_model import ARIMA 

from sklearn.preprocessing import StandardScaler
import os

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Weather 추가변수
- 나중에 모델 돌릴때마다 df에 추가하는 것

In [3]:
# hourly WEATHER data
# w_hr = pd.read_csv("/kaggle/input/weather/hourly.csv",encoding='euc-kr', sep=',')
w_hr = pd.read_csv("data/weather/hourly.csv", sep=',', encoding='euc-kr')
w_hr2 = w_hr.loc[:,['Time', 'Temp','Humidity', 'WindSpeed']]
w_hr2['Humidity']=w_hr2.loc[:,'Humidity'].fillna(0)
w_hr2.head(2)

,Time,Temp,Humidity,WindSpeed
0,2016.7.20 0:00,25.7,0.0,1.3
1,2016.7.20 1:00,25.6,0.0,1.4


In [4]:
w_hr2['Time'] = pd.to_datetime(w_hr2.Time)
w_hr2 = w_hr2.set_index('Time')
w_hr2.head(2)

,Temp,Humidity,WindSpeed
Time,,,
2016-07-20 00:00:00,25.7,0.0,1.3
2016-07-20 01:00:00,25.6,0.0,1.4


## 파생 변수

In [61]:
# test = pd.read_csv("/kaggle/input/dataset/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')
test = pd.read_csv("data/TEST_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')
train = pd.read_csv("data/TRAIN_DHweightDaconArima_NAFilled.csv", index_col=0, sep=',')

In [62]:
test.index = pd.to_datetime(test.index)
train.index = pd.to_datetime(train.index)

In [7]:
def dateTimeVar(df):
    df1 = pd.DataFrame(df.index)
    df1.index = pd.to_datetime(df1.iloc[:,0])
    df1 = df1.drop(columns=df1.columns)
    # datetime을 사용한 파생변수
    # 연도, 월, 일, 요일, 시간
    df1['Mth']= df1.index.month
    df1['Date']=df1.index.day
    df1['Day']=df1.index.weekday
    df1['hour']=df1.index.hour
    
    # Season 1: 봄 ~ 4: 겨울 
    df1['season'] = df1.Mth.apply(lambda x: 4 if x in [12,1,2] else 
                1 if x in [3,4,5] else 2 if x in [6,7,8]
               else 3)
    # dayClass 1: 월, 2: 화~금, 3: 토, 일 
    # 참조 from 실시간 수요예측 기법 개발 및 온라인 수요예측 방안 연구_KPX 한국전력거래소)
    df1['dayClass'] = df1.Day.apply(lambda x: 1 if x ==0 else 
                2 if x in [1,2,3,4] else 3)
    return df1

In [8]:
TE_time_vars = dateTimeVar(test)

In [9]:
TE_time_vars.head(5)

,Mth,Date,Day,hour,season,dayClass
Time,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2
2017-08-29 11:00:00,8,29,1,11,2,2
2017-08-29 12:00:00,8,29,1,12,2,2
2017-08-29 13:00:00,8,29,1,13,2,2
2017-08-29 14:00:00,8,29,1,14,2,2


## Make DUMMIES

In [10]:
test.head(2)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667
2017-08-29 11:00:00,1.458903,0.398,0.293,0.358,0.350,0.232,0.253,0.260,0.168,0.084,...,0.163,0.086,0.288,0.342010,1.150483,0.196,0.343,1.109,0.225,0.29667


In [11]:
TE_time_vars.head(2)

,Mth,Date,Day,hour,season,dayClass
Time,,,,,,
2017-08-29 10:00:00,8,29,1,10,2,2
2017-08-29 11:00:00,8,29,1,11,2,2


In [12]:
def dummies(df, dum_lst):
    df1 = df.copy()
    # dum_lst에 들어있는 각각의 변수마다 
    # tmp = get_dummies, 해당 original 변수는 drop
    # tmp와 변수가 drop된 df를 join 후 리턴
    for v in dum_lst:
        tmp = pd.get_dummies(df1[v], prefix=v)
        df1 = df1.drop(columns=v).join(tmp)
    return df1

In [13]:
TE_timeV_dum = dummies(TE_time_vars, ['season', 'dayClass'])

In [14]:
TE_timeV_dum_wthr= pd.concat([TE_timeV_dum, w_hr2], axis=1,join='inner')

In [15]:
TE_timeV_dum_wthr.head(10)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
Time,,,,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0,19.3,0.0,5.0
2017-08-29 11:00:00,8,29,1,11,0,1,0,0,0,1,0,19.7,0.0,4.7
2017-08-29 12:00:00,8,29,1,12,0,1,0,0,0,1,0,19.4,0.0,4.8
2017-08-29 13:00:00,8,29,1,13,0,1,0,0,0,1,0,19.8,0.0,4.5
2017-08-29 14:00:00,8,29,1,14,0,1,0,0,0,1,0,20.5,0.0,3.4
2017-08-29 15:00:00,8,29,1,15,0,1,0,0,0,1,0,20.6,0.0,4.0
2017-08-29 16:00:00,8,29,1,16,0,1,0,0,0,1,0,22.2,0.0,5.2
2017-08-29 17:00:00,8,29,1,17,0,1,0,0,0,1,0,21.7,0.0,5.5
2017-08-29 18:00:00,8,29,1,18,0,1,0,0,0,1,0,21.2,0.0,5.3


# Modeling

* test : TEST셋의 순수한 meterIDs Time series (NA값 채워짐) 
* TE_timeV_dum: TEST셋의 시간파생변수 + dummy variables 

In [16]:
test.head(1)

,X26,X303,X241,X435,X402,X352,X305,X350,X326,X299,...,X283,X329,X223,X266,X20,X443,X347,X75,X107,X230
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,1.453803,0.431,0.276,0.228,0.432,0.232,0.348,0.259,0.194,0.086,...,0.114,0.087,0.283,0.342142,1.148489,0.094,0.351,1.109,0.225,0.29667


In [17]:
TE_timeV_dum_wthr.head(1)

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
Time,,,,,,,,,,,,,,
2017-08-29 10:00:00,8,29,1,10,0,1,0,0,0,1,0,19.3,0.0,5.0


<br><br><br>


## Random sample 된 30개만 돌리기 (06.20~30 예측)

In [58]:
rs = pd.read_csv("data/train_all.csv", index_col=0, sep=',')
ids = rs.columns

In [59]:

test.loc[:, ids]

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,,,,,,,,,,,,
2017-08-29 10:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.167430,4.171678,3.498120,3.816439,2.518855,0.216611,0.602,0.255000,0.214000,0.332322
2017-08-29 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.159356,4.122120,3.461189,3.801315,2.519404,0.216574,0.602,0.428000,0.256000,0.332338
2017-08-29 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.152415,4.083238,3.430963,3.789355,2.519835,0.216555,0.602,0.387000,0.216000,0.332353
2017-08-29 13:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.146449,4.052734,3.406225,3.779896,2.520170,0.216545,0.602,0.229000,0.215000,0.332367
2017-08-29 14:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.141321,4.028802,3.385979,3.772415,2.520428,0.216541,0.602,0.232000,0.302000,0.332380
2017-08-29 15:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.136913,4.010025,3.369409,3.766499,2.520624,0.216540,0.602,0.239000,0.346000,0.332393
2017-08-29 16:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.133123,3.995295,3.355848,3.761820,2.520770,0.216540,0.602,0.300000,0.284000,0.332406
2017-08-29 17:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.129866,3.983738,3.344749,3.758120,2.520875,0.216542,0.602,0.321000,0.232000,0.332419
2017-08-29 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.127067,3.974671,3.335665,3.755194,2.520947,0.216543,0.602,0.319000,0.202000,0.332432


In [19]:
rs.index = pd.to_datetime(rs.index)

In [31]:
train = rs.loc[:'2018-06-20 23:00:00', :]
train.tail()

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,,,,,,,,,,,,
2018-06-20 19:00:00,9.955,4.105,0.087,1.143,3.143,3.677,7.841,1.065,11.910,6.461,...,2.414,8.631,8.506,3.980,2.469,0.278,0.277,0.527,0.356,0.245
2018-06-20 20:00:00,11.330,4.830,0.087,1.526,1.513,3.682,7.808,6.867,9.281,6.689,...,2.337,8.748,8.429,5.499,2.488,0.331,0.258,0.683,0.500,0.329
2018-06-20 21:00:00,11.256,3.793,0.086,1.438,1.491,3.500,7.949,6.919,9.389,6.873,...,2.406,6.073,6.237,6.931,2.526,0.287,0.245,0.547,0.585,0.793
2018-06-20 22:00:00,9.903,0.304,0.089,1.282,1.506,0.625,2.528,5.521,5.550,6.526,...,2.537,5.168,5.842,8.690,2.534,0.246,0.287,0.403,0.368,0.317
2018-06-20 23:00:00,11.391,0.289,0.103,0.309,1.488,0.305,0.181,5.800,2.600,6.869,...,2.762,5.140,3.603,5.213,2.541,0.183,0.234,0.352,0.329,0.224


In [32]:
features = TE_timeV_dum_wthr.loc[:'2018-06-20 23:00:00', :]
features.tail()

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
Time,,,,,,,,,,,,,,
2018-06-20 19:00:00,6,20,2,19,0,1,0,0,0,1,0,NaN,0.0,3.5
2018-06-20 20:00:00,6,20,2,20,0,1,0,0,0,1,0,NaN,0.0,2.1
2018-06-20 21:00:00,6,20,2,21,0,1,0,0,0,1,0,19.2,0.0,4.5
2018-06-20 22:00:00,6,20,2,22,0,1,0,0,0,1,0,19.0,0.0,4.2
2018-06-20 23:00:00,6,20,2,23,0,1,0,0,0,1,0,19.2,0.0,3.3


In [27]:
test_real = rs.loc['2018-06-21 00:00:00':, :]
test_real.head()

,X40,X84,X77,X93,X89,X15,X23,X60,X27,X6,...,X41,X50,X30,X37,X18,X231,X116,X149,X371,X218
Time,,,,,,,,,,,,,,,,,,,,,
2018-06-21 00:00:00,10.500,0.278,0.102,0.363,1.434,0.334,0.314,5.918,1.828,7.190,...,2.752,5.121,0.668,5.521,2.575,0.145,0.288,0.415,0.329,0.157
2018-06-21 01:00:00,9.905,0.275,0.098,0.360,1.416,1.332,0.298,4.968,1.032,6.841,...,2.723,5.077,0.389,5.110,2.591,0.143,0.172,0.338,0.325,0.191
2018-06-21 02:00:00,9.261,0.233,0.086,0.347,1.398,1.346,0.154,0.908,0.992,6.648,...,2.749,4.136,0.417,3.060,2.563,0.133,0.160,0.341,0.306,0.162
2018-06-21 03:00:00,9.648,0.191,0.086,0.354,1.383,1.352,0.417,1.079,0.961,6.520,...,2.731,3.142,0.412,2.287,2.568,0.132,0.204,0.382,0.297,0.150
2018-06-21 04:00:00,3.945,0.191,0.086,0.388,1.426,1.337,0.216,1.235,0.948,6.645,...,2.837,3.148,0.375,2.717,2.572,0.144,0.189,0.401,0.287,0.221


In [28]:
X_test = TE_timeV_dum_wthr.loc['2018-06-21 00:00:00':, :]
X_test.head()

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed
Time,,,,,,,,,,,,,,
2018-06-21 00:00:00,6,21,3,0,0,1,0,0,0,1,0,19.2,0.0,2.4
2018-06-21 01:00:00,6,21,3,1,0,1,0,0,0,1,0,19.0,0.0,2.5
2018-06-21 02:00:00,6,21,3,2,0,1,0,0,0,1,0,19.1,0.0,1.4
2018-06-21 03:00:00,6,21,3,3,0,1,0,0,0,1,0,19.1,0.0,1.1
2018-06-21 04:00:00,6,21,3,4,0,1,0,0,0,1,0,19.0,0.0,1.9


In [29]:
X_test.shape

(240, 14)

In [36]:
features.shape

(7094, 14)

In [37]:
mID = 'X40'
data = pd.concat([train.loc[:,[mID]], features], axis=1)
data = data.rename(columns={mID:'y'})
for i in [24, 168]:
    data['lag_{}'.format(i)] = data.y.shift(i)
y_train = data.dropna().y
X_train = data.dropna().drop(['y'], axis=1)

X_train

,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed,lag_24,lag_168
Time,,,,,,,,,,,,,,,,
2018-02-22 00:00:00,2,22,3,0,0,0,0,1,0,1,0,-1.1,0.0,2.4,10.592437,10.592437
2018-02-22 01:00:00,2,22,3,1,0,0,0,1,0,1,0,-1.3,0.0,2.0,10.592437,10.592437
2018-02-22 02:00:00,2,22,3,2,0,0,0,1,0,1,0,-1.6,0.0,2.0,10.592437,10.592437
2018-02-22 03:00:00,2,22,3,3,0,0,0,1,0,1,0,-1.0,0.0,3.0,10.592437,10.592437
2018-02-22 04:00:00,2,22,3,4,0,0,0,1,0,1,0,-1.2,0.0,3.2,10.592437,10.592437
2018-02-22 05:00:00,2,22,3,5,0,0,0,1,0,1,0,-1.6,0.0,3.5,10.592437,10.592437
2018-02-22 06:00:00,2,22,3,6,0,0,0,1,0,1,0,-2.0,0.0,3.6,10.592437,10.592437
2018-02-22 07:00:00,2,22,3,7,0,0,0,1,0,1,0,-2.1,0.0,3.6,10.592437,10.592437
2018-02-22 08:00:00,2,22,3,8,0,0,0,1,0,1,0,-1.9,0.0,3.7,10.592437,10.592437


In [54]:
data.iloc[-168:,:] data.

,y,Mth,Date,Day,hour,season_1,season_2,season_3,season_4,dayClass_1,dayClass_2,dayClass_3,Temp,Humidity,WindSpeed,lag_24,lag_168
Time,,,,,,,,,,,,,,,,,
2018-06-14 00:00:00,14.083,6,14,3,0,0,1,0,0,0,1,0,20.2,0.0,1.2,11.288,13.249
2018-06-14 01:00:00,14.528,6,14,3,1,0,1,0,0,0,1,0,20.3,0.0,2.0,10.860,11.587
2018-06-14 02:00:00,12.592,6,14,3,2,0,1,0,0,0,1,0,20.6,0.0,0.8,8.531,11.984
2018-06-14 03:00:00,11.551,6,14,3,3,0,1,0,0,0,1,0,20.5,0.0,2.6,9.508,10.533
2018-06-14 04:00:00,3.210,6,14,3,4,0,1,0,0,0,1,0,19.9,1.6,1.2,3.612,3.635
2018-06-14 05:00:00,2.898,6,14,3,5,0,1,0,0,0,1,0,19.2,6.5,1.5,2.861,3.477
2018-06-14 06:00:00,2.672,6,14,3,6,0,1,0,0,0,1,0,18.8,3.0,2.0,2.962,3.344
2018-06-14 07:00:00,3.506,6,14,3,7,0,1,0,0,0,1,0,18.3,2.2,3.9,4.527,4.742
2018-06-14 08:00:00,10.163,6,14,3,8,0,1,0,0,0,1,0,18.0,6.5,3.0,10.207,10.858


In [57]:
[data.iloc[-168:,0].values, data.iloc[-168:-96,0].values]

[array([14.083, 14.528, 12.592, 11.551,  3.21 ,  2.898,  2.672,  3.506,
        10.163, 10.159, 10.239, 10.478,  9.951,  9.844, 10.743,  9.991,
        10.369, 13.66 , 16.504, 15.298, 15.253, 15.784, 14.767, 17.093,
        15.376, 14.33 , 12.952, 12.24 ,  4.035,  3.2  ,  3.284,  3.646,
        10.671, 11.893, 11.533, 10.836, 10.318, 12.284, 14.025, 14.962,
        14.589, 15.062, 15.112, 14.75 , 15.338, 16.535, 16.529, 14.18 ,
        17.403, 18.255, 14.229, 11.879,  3.943,  3.257,  3.266,  3.927,
        10.469, 10.19 , 10.091, 11.959, 15.293, 14.543, 14.618, 16.954,
        16.849, 16.133,    nan,    nan,    nan,    nan,    nan,    nan,
        14.915,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
           nan,    nan,    nan,    nan,    nan,    nan,    nan,    nan,
        11.588,  8.222,  8.222,  8.222,  8.222,  8.222,  8.222,  8.222,
         8.222,  9.574,  9.61 , 10.547, 16.553, 15.489, 15.017, 

In [34]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

lg_param = {"boosting_type":['gbdt', 'dart','goss'],
    "learning_rate": [0.01,0.05,0.1,0.15],
    "min_child_samples":[10, 20, 30, 50],       
    "n_estimators":[10, 50, 100],
    }

def gsLGBR_Hrly_FitPred (model, params, mID, X_train, y_train, X_test):
    grid_gb = GridSearchCV(model, params, cv=4, verbose=5)
    grid_gb.fit(X_train, y_train)
    fcst = grid_gb.predict(X_test)
    a = pd.DataFrame([mID], columns=['meter_id'])
    for i in range(240):
        a[i+1]=[fcst[i]]
    return a
   
def onlytesting(train, features, X_test):
    hrly = pd.DataFrame()
    for mID in train.columns:
        data = pd.concat([train.loc[:,[mID]], features], axis=1)
        data = data.rename(columns={mID:'y'})
        for i in [24, 168]:
            data['lag_{}'.format(i)] = data.y.shift(i)
        y_train = data.dropna().y
        X_train = data.dropna().drop(['y'], axis=1)
        
        X_test['lag_24'] = y_train.tail(24).values
        X_test['lag_168'] = data.iloc[-168:,:].values
        
        model = LGBMRegressor(verbose=1, tree_learner='feature')
        tmp = gsLGBR_Hrly_FitPred(model, lg_param,
                        mID, X_train, y_train, X_test_H)
        hrly = pd.concat([hrly_50, tmp], axis=0)
    return hrly

In [33]:
train2 = train.iloc[:,0:2]
train2.head(2)

,X40,X84
Time,,
2018-02-15 00:00:00,10.592437,0.164
2018-02-15 01:00:00,10.592437,0.161


In [35]:
trial = onlytesting(train2, features, X_test)

NameError: name 'X_test_H' is not defined